<a href="https://colab.research.google.com/github/FaizFP/Dicoding--Analystic-Sentiments/blob/main/Pelatihan_SentimenAnalys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
!pip install gensim

from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from collections import Counter




In [3]:
nama_file = '/content/sample_data/dataset_final.csv'
df_bersih = pd.read_csv(nama_file)
print(df_bersih.head())

                                       clean_content sentimen
0  aplikasi bantu order gojek berangkat pulang kerja  positif
1  drivernya ambil pas hujan ga kaya grab yg maha...  positif
2                                  wow uda tahin aja  positif
3                                               good  positif
4  kecewa bgt ga segercep grab pake gojek kapok b...  negatif


In [4]:
df_bersih.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_content  10692 non-null  object
 1   sentimen       10692 non-null  object
dtypes: object(2)
memory usage: 167.2+ KB


In [5]:
df_bersih['clean_content'].isnull().sum()

0

In [6]:
df_bersih['sentimen'].value_counts()

,count
sentimen,
positif,7372
negatif,2916
netral,404


In [7]:
df_bersih.duplicated().sum()

3415

In [8]:
df_bersih['sentimen'].value_counts()

,count
sentimen,
positif,7372
negatif,2916
netral,404


In [9]:

# 1. Inisialisasi TfidfVectorizer
# max_features=5000 berarti kita hanya akan mengambil 5000 kata/fitur yang paling penting
# Ini membantu model fokus pada kata-kata yang paling relevan dan mempercepat training
vectorizer = TfidfVectorizer(max_features=5000)

# 2. Definisikan variabel fitur (X) dan target (y)
# Fitur (X) adalah teks ulasan yang sudah bersih
# Target (y) adalah label sentimen yang sudah kita buat
X_text = df_bersih['clean_content']
y = df_bersih['sentimen']

# 3. Lakukan proses fit_transform
# 'fit' mempelajari kosakata dari data teks Anda
# 'transform' mengubah teks tersebut menjadi matriks angka TF-IDF
print("Memulai proses ekstraksi fitur dengan TF-IDF...")
X = vectorizer.fit_transform(X_text)
print("Ekstraksi fitur selesai.")

# 4. Periksa hasil dari proses ini
print(f"\nDimensi dari matriks fitur (X): {X.shape}")
print(f"Dimensi dari target (y): {y.shape}")

Memulai proses ekstraksi fitur dengan TF-IDF...
Ekstraksi fitur selesai.

Dimensi dari matriks fitur (X): (10692, 5000)
Dimensi dari target (y): (10692,)


## 🧪 Skema 1: SVM + TF-IDF (Data 80/20)

*   Item daftar
*   Item daftar




In [10]:
# 1. Pembagian Data (80% Latih, 20% Uji)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 2. Pelatihan Model SVM
print("Memulai pelatihan model SVM...")
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)
print("Pelatihan SVM selesai.")

# 3. Evaluasi Model
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print(f"\n--- Hasil Skema 1: SVM + TF-IDF (80/20) ---")
print(f"Akurasi: {accuracy_svm * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred_svm))

Memulai pelatihan model SVM...
Pelatihan SVM selesai.

--- Hasil Skema 1: SVM + TF-IDF (80/20) ---
Akurasi: 88.17%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.79      0.82      0.80       583
      netral       0.00      0.00      0.00        81
     positif       0.92      0.95      0.94      1475

    accuracy                           0.88      2139
   macro avg       0.57      0.59      0.58      2139
weighted avg       0.85      0.88      0.86      2139



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Tuning

In [11]:
param_grid_expanded = [
    {
        'kernel': ['linear'],
        'C': [1, 10, 100]
    },
    {
        'kernel': ['rbf'],
        'C': [1, 10, 100],
        'gamma': [0.1, 0.01, 0.001]
    }
]

# 2. Inisialisasi GridSearchCV dengan param_grid yang baru
# Perhatikan kita tidak lagi mendefinisikan kernel di dalam SVC()
grid_search_expanded = GridSearchCV(
    SVC(random_state=42), # Model dasar tanpa kernel spesifik
    param_grid_expanded,  # Gunakan grid parameter yang sudah diperluas
    cv=3,                 # cv=3 untuk mempercepat pencarian yang lebih kompleks ini
    n_jobs=-1,
    verbose=2
)

# 3. Jalankan pencarian pada data latih Anda (X_train dan y_train)
print("Memulai pencarian hyperparameter yang diperluas (kernel, C, gamma)...")
grid_search_expanded.fit(X_train, y_train)
print("Pencarian selesai.")

# 4. Lihat kombinasi parameter terbaik yang ditemukan
print(f"\nParameter terbaik yang ditemukan: {grid_search_expanded.best_params_}")
print(f"Skor cross-validation terbaik: {grid_search_expanded.best_score_:.4f}")


# 5. Gunakan model terbaik untuk membuat prediksi
best_model_overall = grid_search_expanded.best_estimator_
y_pred_best = best_model_overall.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

# 6. Tampilkan hasil akurasi yang sudah ditingkatkan
print(f"\n--- Hasil Terbaik Setelah Pencarian Diperluas ---")
print(f"Akurasi: {accuracy_best * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred_best))

Memulai pencarian hyperparameter yang diperluas (kernel, C, gamma)...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Pencarian selesai.

Parameter terbaik yang ditemukan: {'C': 1, 'kernel': 'linear'}
Skor cross-validation terbaik: 0.8741

--- Hasil Terbaik Setelah Pencarian Diperluas ---
Akurasi: 88.17%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.79      0.82      0.80       583
      netral       0.00      0.00      0.00        81
     positif       0.92      0.95      0.94      1475

    accuracy                           0.88      2139
   macro avg       0.57      0.59      0.58      2139
weighted avg       0.85      0.88      0.86      2139



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 🌳 Skema 2: Random Forest + Word2Vec (Data 80/20)


In [12]:

# A. Tokenisasi data teks (Word2Vec butuh input berupa list of list of words)
tokenized_text = [text.split() for text in df_bersih['clean_content']]

# B. Latih model Word2Vec
# size: dimensi vektor, window: jarak kata, min_count: abaikan kata yg jarang muncul
print("Memulai pelatihan model Word2Vec...")
w2v_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
print("Pelatihan Word2Vec selesai.")

# C. Ubah setiap ulasan menjadi satu vektor rata-rata
def vectorize_review(review, model):
    words = review.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv.key_to_index]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

X_w2v = np.array([vectorize_review(review, w2v_model) for review in df_bersih['clean_content']])
y_w2v = df_bersih['sentimen']

print(f"Dimensi matriks fitur Word2Vec (X_w2v): {X_w2v.shape}")

Memulai pelatihan model Word2Vec...
Pelatihan Word2Vec selesai.
Dimensi matriks fitur Word2Vec (X_w2v): (10692, 100)


In [13]:

# 1. Pembagian Data (80% Latih, 20% Uji)
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(
    X_w2v, y_w2v,
    test_size=0.2,
    random_state=42,
    stratify=y_w2v
)

# 2. Pelatihan Model Random Forest
print("Memulai pelatihan model Random Forest...")
rf_model_w2v = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_w2v.fit(X_train_w2v, y_train_w2v)
print("Pelatihan Random Forest selesai.")

# 3. Evaluasi Model
y_pred_rf_w2v = rf_model_w2v.predict(X_test_w2v)
accuracy_rf_w2v = accuracy_score(y_test_w2v, y_pred_rf_w2v)

print(f"\n--- Hasil Skema 2: RF + Word2Vec (80/20) ---")
print(f"Akurasi: {accuracy_rf_w2v * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_w2v, y_pred_rf_w2v))

Memulai pelatihan model Random Forest...
Pelatihan Random Forest selesai.

--- Hasil Skema 2: RF + Word2Vec (80/20) ---
Akurasi: 83.50%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.71      0.72      0.72       583
      netral       0.00      0.00      0.00        81
     positif       0.88      0.92      0.90      1475

    accuracy                           0.83      2139
   macro avg       0.53      0.55      0.54      2139
weighted avg       0.80      0.83      0.82      2139



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##TUNING KE 85

In [14]:
# 1. Definisikan parameter yang ingin diuji
# Kita akan menguji beberapa pengaturan umum DAN parameter class_weight
param_grid_rf_w2v = {
    'n_estimators': [100, 200],              # Jumlah pohon
    'max_depth': [10, 20, None],             # Kedalaman maksimum pohon
    'min_samples_leaf': [1, 2],              # Jumlah minimum sampel di daun
    'class_weight': [None, 'balanced']       # <-- PENTING: Uji dengan dan tanpa bobot kelas
}

# 2. Inisialisasi GridSearchCV
grid_search_rf_w2v = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid_rf_w2v,
    cv=3,  # cv=3 untuk mempercepat proses pencarian
    n_jobs=-1,
    verbose=2
)

# 3. Jalankan pencarian pada data latih Anda
print("Memulai pencarian hyperparameter terbaik untuk Random Forest + Word2Vec...")
grid_search_rf_w2v.fit(X_train_w2v, y_train_w2v)
print("Pencarian selesai.")

# 4. Lihat parameter terbaik yang ditemukan
print(f"\nParameter terbaik yang ditemukan: {grid_search_rf_w2v.best_params_}")

# 5. Gunakan model terbaik untuk membuat prediksi
best_rf_model_w2v = grid_search_rf_w2v.best_estimator_
y_pred_best_rf_w2v = best_rf_model_w2v.predict(X_test_w2v)
accuracy_best_rf_w2v = accuracy_score(y_test_w2v, y_pred_best_rf_w2v)

# 6. Tampilkan hasil akurasi yang sudah ditingkatkan
print(f"\n--- Hasil Skema 2 (Setelah Tuning) ---")
print(f"Akurasi: {accuracy_best_rf_w2v * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_w2v, y_pred_best_rf_w2v))

Memulai pencarian hyperparameter terbaik untuk Random Forest + Word2Vec...
Fitting 3 folds for each of 24 candidates, totalling 72 fits
Pencarian selesai.

Parameter terbaik yang ditemukan: {'class_weight': None, 'max_depth': None, 'min_samples_leaf': 2, 'n_estimators': 100}

--- Hasil Skema 2 (Setelah Tuning) ---
Akurasi: 83.40%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.71      0.73      0.72       583
      netral       0.00      0.00      0.00        81
     positif       0.88      0.92      0.90      1475

    accuracy                           0.83      2139
   macro avg       0.53      0.55      0.54      2139
weighted avg       0.80      0.83      0.82      2139



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
# ==============================================================================
# Tahap 1: Buat Bobot TF-IDF untuk Setiap Kata
# ==============================================================================

print("Melatih TfidfVectorizer untuk mendapatkan bobot kata...")
tfidf_vectorizer = TfidfVectorizer()
# Kita hanya perlu 'fit' untuk mempelajari kosakatanya
tfidf_vectorizer.fit(df_bersih['clean_content'])
# Buat dictionary yang memetakan setiap kata ke bobot IDF-nya
word_idf_weights = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_))


# ==============================================================================
# Tahap 2: Buat Fungsi Vektorisasi Baru yang Menggunakan Bobot TF-IDF
# ==============================================================================

def vectorize_review_weighted(review, w2v_model, idf_weights):
    words = review.split()
    word_vectors = []

    for word in words:
        # Cek apakah kata ada di model Word2Vec DAN di kamus IDF
        if word in w2v_model.wv and word in idf_weights:
            # Kalikan vektor kata dengan bobot IDF-nya
            # Ini memberikan "kepentingan" lebih pada kata-kata yang lebih unik
            word_vectors.append(w2v_model.wv[word] * idf_weights[word])

    if not word_vectors:
        # Jika tidak ada kata yang valid, kembalikan vektor nol
        return np.zeros(w2v_model.vector_size)

    # Rata-ratakan vektor yang sudah diboboti
    return np.mean(word_vectors, axis=0)


# ==============================================================================
# Tahap 3: Buat Fitur Baru, Latih, dan Evaluasi Model
# ==============================================================================

print("\nMembuat fitur TF-IDF Weighted Word2Vec...")
# Terapkan fungsi baru untuk membuat matriks fitur yang lebih baik
X_w2v_weighted = np.array([vectorize_review_weighted(r, w2v_model, word_idf_weights) for r in df_bersih['clean_content']])
y_w2v_weighted = df_bersih['sentimen']

# Pembagian Data
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(
    X_w2v_weighted, y_w2v_weighted, test_size=0.2, random_state=42, stratify=y_w2v_weighted
)

# Inisialisasi dan Latih Model Random Forest
rf_model_w2v_weighted = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

print("\nMemulai pelatihan Random Forest dengan fitur baru yang telah ditingkatkan...")
rf_model_w2v_weighted.fit(X_train_w, y_train_w)
print("Pelatihan selesai.")

# Evaluasi Model
y_pred_w = rf_model_w2v_weighted.predict(X_test_w)
accuracy_w = accuracy_score(y_test_w, y_pred_w)

print(f"\n--- Hasil Skema 2 (dengan TF-IDF Weighted Word2Vec) ---")
print(f"Akurasi: {accuracy_w * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_w, y_pred_w))

Melatih TfidfVectorizer untuk mendapatkan bobot kata...

Membuat fitur TF-IDF Weighted Word2Vec...

Memulai pelatihan Random Forest dengan fitur baru yang telah ditingkatkan...
Pelatihan selesai.

--- Hasil Skema 2 (dengan TF-IDF Weighted Word2Vec) ---
Akurasi: 83.87%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.72      0.74      0.73       583
      netral       0.00      0.00      0.00        81
     positif       0.88      0.93      0.90      1475

    accuracy                           0.84      2139
   macro avg       0.53      0.55      0.54      2139
weighted avg       0.81      0.84      0.82      2139



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
# ==============================================================================
# Tahap 1: Latih Ulang Model Word2Vec dengan Parameter yang Ditingkatkan
# ==============================================================================
print("Memulai pelatihan model Word2Vec yang ditingkatkan...")
w2v_model_tuned = Word2Vec(
    sentences=tokenized_text,
    vector_size=200,  # <-- Ditingkatkan dari 100 menjadi 200
    window=10,        # <-- Ditingkatkan dari 5 menjadi 10
    min_count=2,      # <-- Abaikan kata yang hanya muncul 1x (mengurangi noise)
    sg=1,             # <-- Gunakan algoritma Skip-gram
    workers=4
)
print("Pelatihan Word2Vec selesai.")

# ==============================================================================
# Tahap 2: Buat Ulang Fitur Dokumen dan Latih Model RF
# ==============================================================================

# Fungsi untuk membuat vektor rata-rata (tetap sama)
def vectorize_review(review, model):
    words = review.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Buat fitur baru dengan model Word2Vec yang sudah di-tuning
print("\nMembuat fitur dokumen baru dengan Word2Vec yang ditingkatkan...")
X_w2v_tuned = np.array([vectorize_review(review, w2v_model_tuned) for review in df_bersih['clean_content']])
y_w2v_tuned = df_bersih['sentimen']

# Pembagian Data
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
    X_w2v_tuned, y_w2v_tuned, test_size=0.2, random_state=42, stratify=y_w2v_tuned
)

# Latih model RF (kita bisa langsung gunakan class_weight='balanced' karena itu memberi sedikit kenaikan)
rf_model_tuned_w2v = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

print("\nMemulai pelatihan Random Forest dengan fitur baru...")
rf_model_tuned_w2v.fit(X_train_t, y_train_t)
print("Pelatihan selesai.")

# ==============================================================================
# Tahap 3: Evaluasi Hasil Akhir
# ==============================================================================
y_pred_tuned = rf_model_tuned_w2v.predict(X_test_t)
accuracy_tuned = accuracy_score(y_test_t, y_pred_tuned)

print(f"\n--- Hasil Skema 2 (Setelah Tuning Word2Vec) ---")
print(f"Akurasi: {accuracy_tuned * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_t, y_pred_tuned))

Memulai pelatihan model Word2Vec yang ditingkatkan...
Pelatihan Word2Vec selesai.

Membuat fitur dokumen baru dengan Word2Vec yang ditingkatkan...

Memulai pelatihan Random Forest dengan fitur baru...
Pelatihan selesai.

--- Hasil Skema 2 (Setelah Tuning Word2Vec) ---
Akurasi: 85.51%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.73      0.81      0.77       583
      netral       0.00      0.00      0.00        81
     positif       0.91      0.92      0.92      1475

    accuracy                           0.86      2139
   macro avg       0.55      0.58      0.56      2139
weighted avg       0.83      0.86      0.84      2139



## 🌲 Skema 3: Random Forest + TF-IDF (Data 70/30)


In [17]:
# 1. Pembagian Data (70% Latih, 30% Uji)
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X, y,
    test_size=0.3, # Perhatikan test_size diubah menjadi 0.3
    random_state=42,
    stratify=y
)

# 2. Pelatihan Model Random Forest
print("Memulai pelatihan model Random Forest...")
rf_model_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_tfidf.fit(X_train_rf, y_train_rf)
print("Pelatihan Random Forest selesai.")

# 3. Evaluasi Model
y_pred_rf_tfidf = rf_model_tfidf.predict(X_test_rf)
accuracy_rf_tfidf = accuracy_score(y_test_rf, y_pred_rf_tfidf)

print(f"\n--- Hasil Skema 3: RF + TF-IDF (70/30) ---")
print(f"Akurasi: {accuracy_rf_tfidf * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_rf, y_pred_rf_tfidf))

Memulai pelatihan model Random Forest...
Pelatihan Random Forest selesai.

--- Hasil Skema 3: RF + TF-IDF (70/30) ---
Akurasi: 87.03%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.74      0.85      0.79       875
      netral       0.00      0.00      0.00       121
     positif       0.93      0.93      0.93      2212

    accuracy                           0.87      3208
   macro avg       0.56      0.59      0.57      3208
weighted avg       0.84      0.87      0.86      3208



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
# Inisialisasi model dengan parameter terbaik DAN class_weight
rf_model_balanced = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_leaf=1,
    class_weight='balanced', # <-- Tambahkan parameter penting ini
    random_state=42,
    n_jobs=-1
)

print("Memulai pelatihan Random Forest dengan class_weight='balanced'...")
rf_model_balanced.fit(X_train_rf, y_train_rf)
print("Pelatihan selesai.")

# Evaluasi Model
y_pred_balanced_rf = rf_model_balanced.predict(X_test_rf)
accuracy_balanced_rf = accuracy_score(y_test_rf, y_pred_balanced_rf)

print(f"\n--- Hasil Skema 3 (dengan Class Weight Balanced) ---")
print(f"Akurasi: {accuracy_balanced_rf * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_rf, y_pred_balanced_rf))

Memulai pelatihan Random Forest dengan class_weight='balanced'...
Pelatihan selesai.

--- Hasil Skema 3 (dengan Class Weight Balanced) ---
Akurasi: 86.91%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.75      0.84      0.79       875
      netral       0.00      0.00      0.00       121
     positif       0.93      0.93      0.93      2212

    accuracy                           0.87      3208
   macro avg       0.56      0.59      0.57      3208
weighted avg       0.84      0.87      0.86      3208



In [19]:
# 1. Definisikan parameter yang ingin diuji
# Ini adalah beberapa pengaturan umum yang bagus untuk memulai
param_grid_rf = {
    'n_estimators': [100, 200],      # Coba dengan 100 atau 200 pohon
    'max_depth': [None, 10, 20],     # Coba tanpa batas kedalaman, atau kedalaman 10 dan 20
    'min_samples_leaf': [1, 2],      # Coba dengan minimal 1 atau 2 sampel di daun
}

# 2. Inisialisasi GridSearchCV
grid_search_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    cv=3,  # cv=3 untuk mempercepat proses pencarian
    n_jobs=-1,
    verbose=2
)

# 3. Jalankan pencarian pada data latih Anda
print("Memulai pencarian hyperparameter terbaik untuk Random Forest...")
grid_search_rf.fit(X_train_rf, y_train_rf)
print("Pencarian selesai.")

# 4. Lihat parameter terbaik yang ditemukan
print(f"\nParameter terbaik yang ditemukan: {grid_search_rf.best_params_}")

# 5. Gunakan model terbaik untuk membuat prediksi
best_rf_model = grid_search_rf.best_estimator_
y_pred_best_rf = best_rf_model.predict(X_test_rf)
accuracy_best_rf = accuracy_score(y_test_rf, y_pred_best_rf)

# 6. Tampilkan hasil akurasi yang sudah ditingkatkan
print(f"\n--- Hasil Skema 3 (Setelah Tuning) ---")
print(f"Akurasi: {accuracy_best_rf * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_rf, y_pred_best_rf))

Memulai pencarian hyperparameter terbaik untuk Random Forest...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Pencarian selesai.

Parameter terbaik yang ditemukan: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 200}

--- Hasil Skema 3 (Setelah Tuning) ---
Akurasi: 87.03%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.74      0.84      0.79       875
      netral       0.00      0.00      0.00       121
     positif       0.93      0.93      0.93      2212

    accuracy                           0.87      3208
   macro avg       0.56      0.59      0.57      3208
weighted avg       0.84      0.87      0.86      3208



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 🧪 Skema 1: SVM + TF-IDF (Data 80/20)
Terbagus 88.17

In [21]:
def preprocess_text_cepat(text):
    # 1. Case Folding: Mengubah teks menjadi huruf kecil
    text = text.lower()

    # 2. Punctuation & Number Removal: Menghapus karakter, angka, dan spasi berlebih
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 3. Tokenization: Memecah kalimat menjadi token (kata)
    tokens = word_tokenize(text)

    # 4. Stopword Removal: Menghapus kata-kata umum
    stop_words = set(stopwords.words('indonesian'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Menggabungkan kembali token menjadi kalimat
    return ' '.join(filtered_tokens)

print("Fungsi 'preprocess_text_cepat' berhasil didefinisikan dan siap digunakan.")

Fungsi 'preprocess_text_cepat' berhasil didefinisikan dan siap digunakan.


In [28]:
import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.corpus import stopwords



# --- Langkah 1: Siapkan Data Uji Baru ---
# Ini adalah contoh ulasan baru yang belum pernah dilihat model
ulasan_baru = [
    "Aplikasinya keren dan sangat membantu, drivernya juga ramah banget!",
    "Sering error dan tidak bisa login, sangat mengecewakan pelayanannya.",
    "Biasa saja sih, fiturnya standar tidak ada yang spesial.",
    "kenapa sekarang jadi lambat sekali loadingnya?",
    "terima kasih gojek, promonya mantap"
]

print("Data ulasan baru yang akan dites:")
for ulasan in ulasan_baru:
    print(f"- {ulasan}")


# --- Langkah 2: Lakukan Preprocessing pada Data Baru ---
# Gunakan fungsi preprocessing yang SAMA PERSIS dengan saat training
print("\nMelakukan preprocessing pada data baru...")
ulasan_baru_bersih = [preprocess_text_cepat(ulasan) for ulasan in ulasan_baru]


# --- Langkah 3: Lakukan Ekstraksi Fitur (Transformasi) ---
# PENTING: Gunakan objek 'vectorizer' yang sudah di-fit sebelumnya.
# Gunakan .transform(), JANGAN .fit_transform() lagi.
print("Melakukan ekstraksi fitur (transformasi) pada data baru...")
fitur_ulasan_baru = vectorizer.transform(ulasan_baru_bersih)


# --- Langkah 4: Lakukan Prediksi dengan Model Terbaik Anda ---
print("Melakukan prediksi sentimen...")
prediksi_sentimen = svm_model.predict(fitur_ulasan_baru)


# --- Langkah 5: Tampilkan Hasil Prediksi ---
print("\n--- Hasil Prediksi Sentimen ---")
for i in range(len(ulasan_baru)):
    print(f"Ulasan: \"{ulasan_baru[i]}\"")
    print(f"Prediksi Sentimen: {prediksi_sentimen[i].upper()}")
    print("-" * 30)

Data ulasan baru yang akan dites:
- Aplikasinya keren dan sangat membantu, drivernya juga ramah banget!
- Sering error dan tidak bisa login, sangat mengecewakan pelayanannya.
- Biasa saja sih, fiturnya standar tidak ada yang spesial.
- kenapa sekarang jadi lambat sekali loadingnya?
- terima kasih gojek, promonya mantap

Melakukan preprocessing pada data baru...
Melakukan ekstraksi fitur (transformasi) pada data baru...
Melakukan prediksi sentimen...

--- Hasil Prediksi Sentimen ---
Ulasan: "Aplikasinya keren dan sangat membantu, drivernya juga ramah banget!"
Prediksi Sentimen: POSITIF
------------------------------
Ulasan: "Sering error dan tidak bisa login, sangat mengecewakan pelayanannya."
Prediksi Sentimen: NEGATIF
------------------------------
Ulasan: "Biasa saja sih, fiturnya standar tidak ada yang spesial."
Prediksi Sentimen: POSITIF
------------------------------
Ulasan: "kenapa sekarang jadi lambat sekali loadingnya?"
Prediksi Sentimen: NEGATIF
------------------------------


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
